In [137]:
import pandas as pd
from collections import Counter
import tqdm
import re
import math
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
import functools
import sys
from __future__ import division # for python2 compatability
from nltk.stem.snowball import RussianStemmer

In [2]:
dftrain = pd.read_csv("train_task1_latest.csv", encoding='utf-8')[:-1000]
dfval = pd.read_csv("train_task1_latest.csv", encoding='utf-8')[-1000:]
dftest = pd.read_csv("sdsj_A_test.csv", encoding='utf-8')

In [77]:
stemer = RussianStemmer()
stem_cache = {}

def get_stem(token):
    stem = stem_cache.get(token, None)
    if stem:
        return stem
    token = token.lower()
    stem = stemer.stem(token)
    stem_cache[token] = stem
    return stem

def words_arr(text):
    words = re.findall("\w+", text, re.UNICODE);
    stem_words = []
    for word in words:
        stem_words.append(get_stem(word))
    return stem_words

def uniq_words(text):
    words = re.findall("\w+", text, re.UNICODE);
    stem_words = []
    for word in words:
        stem_words.append(get_stem(word))
    return set(stem_words)

def calculate_idfs(data):
    counter_paragraph = Counter()
    uniq_paragraphs = data['paragraph'].unique()
    for paragraph in tqdm.tqdm(uniq_paragraphs, desc="calc idf"):
        set_words = uniq_words(paragraph)
        counter_paragraph.update(set_words)
        
    num_docs = uniq_paragraphs.shape[0]
    idfs = {}
    for word in counter_paragraph:
        idfs[word] = np.log(num_docs / counter_paragraph[word])
    return idfs

In [247]:
stem_count = Counter()

def count_unique_words(texts):
    for text in texts:
        words = words_arr(text)
        for word in words:
            stem = get_stem(word)
            stem_count[stem] += 1

count_unique_words(dftrain.paragraph)
count_unique_words(dftrain.question)
count_unique_words(dftest.paragraph)
count_unique_words(dftest.question)

print("Total unique stems found: ", len(stem_count))

Total unique stems found:  63784


In [307]:
VOCAB_SKIP_SIZE = 1000
VOCAB_SIZE = len(stem_count) - VOCAB_SKIP_SIZE
vocab = sorted(stem_count, key=stem_count.get, reverse=True)[VOCAB_SKIP_SIZE:]
print(vocab[:100])

['фактор', 'никол', 'потер', 'поиск', 'отказ', 'святосла', 'формир', 'объясн', 'поддержив', 'глебович', 'помог', 'правительств', 'проект', 'торг', 'единиц', 'нормативн', 'пис', 'выбор', '300', 'бабочк', 'советск', 'pistols', 'кредитн', 'преступл', 'жител', 'леж', 'сегодн', 'полет', 'задн', 'опир', 'защищ', 'показател', 'дол', '15', 'подробн', 'процент', 'круг', 'региональн', 'страд', 'москв', 'лучш', 'происхожд', 'сто', 'ричард', 'истин', 'внов', 'запрет', 'дев', 'встрет', 'резерв', 'средневек', 'клеток', 'размнож', 'примитивн', 'принципиальн', '30', 'доступн', 'рома', 'уточн', 'тург', 'прежд', 'участк', 'сближ', 'стоик', 'рамк', 'достоверн', 'ориентиров', 'схем', 'характериз', 'органическ', 'удовлетвор', 'сахар', 'ткан', 'успех', 'операц', 'монет', 'восточн', 'отпра', 'знак', '25', 'выступл', 'полковник', 'сейчас', 'август', '000', 'июн', 'сочета', 'существова', 'доход', 'предста', 'ад', 'усредн', 'перевод', 'процессор', 'and', 'возникл', 'дет', 'контекст', 'обм', 'показ']


In [290]:
word_2_idx = {vocab[i] : i for i in range(VOCAB_SIZE)}

def text_to_vector(text, show_unknowns=False):
    vector = np.zeros(VOCAB_SIZE, dtype=np.int_)
    for word in words_arr(text):
        stem = get_stem(word)
        idx = word_2_idx.get(stem, None)
        if idx is not None:
            vector[idx] = 1
        elif show_unknowns:
            print("Unknown token: {}".format(token))
    return vector

In [291]:
text = dftrain.paragraph[0]
print("text: {}".format(text))
print("vector: {}".format(text_to_vector(text)[:100]))

text: В отличие от рыб, земноводные (амфибии) и пресмыкающиеся (рептилии или гады) уже имеют два круга кровообращения и сердце у них трёхкамерное (появляется межпредсердная перегородка). Единственные современные рептилии, имеющие хотя и неполноценное (межпредсердиевая перегородка не полностью разделяет предсердия, что скорей всего связано с переходом предков к полуводному образу жизни и снижению активности), но уже четырёхкамерное сердце — крокодилы. Считается, что впервые четырёхкамерное сердце появилось у примитивных архозавров и развитых синапсидов. В дальнейшем такое строение сердца унаследовали прямые потомки динозавров — птицы и потомки примитивных млекопитающих — современные млекопитающие.
vector: [1 1 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [292]:
def get_rare_words_score(text1, text2):
    vec1 = text_to_vector(text1)
    vec2 = text_to_vector(text2)
    return sum(np.multiply(vec1, vec2))

In [293]:
text1 = dftrain.paragraph[13] #'Иван родил девченку, велел нести пеленку'
text2 = dftrain.question[13] #'Кого родил Иван?'

def text_arr_to_vec(text_arr, vocab):
    result = []
    for vocab_word in vocab:
        result_word = []
        for word in text_arr:
            if word == vocab_word:
                result_word.append(1)
            else:
                result_word.append(0)
        result.append(result_word)
    return result

def remove_outliers(arr):
    result = []
    for i in range(len(arr)):
        first = 0
        third = 0
        if i > 0:
            first = arr[i-1]
        if i < len(arr) - 1:
            third = arr[i+1]
        if first == 0 and third == 0 and arr[i] <= 1:
            result.append(0)
        else: 
            result.append(arr[i])
    
    return result

def get_convolve_score(text1, text2):
    par = words_arr(text1)
    que = words_arr(text2)
    same_words = uniq_words(text1) &  uniq_words(text2)
    conv1 = text_arr_to_vec(par, list(same_words));
    conv2 = text_arr_to_vec(que, list(same_words));
    convols = [];
    for i in range(len(same_words)):
        convols.append(np.convolve(conv1[i], conv2[i]))
    if len(convols) > 1:
        convols_sum = sum(convols)
    elif len(convols) == 0:
        return 0
    else:
        convols_sum = convols[0]
    return sum(remove_outliers(convols_sum))

print(get_convolve_score(text1, text2))


0


In [294]:
for name, df in [('train', dftrain), ('test', dftest), ('val', dfval)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):
        question = uniq_words(row.question)
        paragraph = uniq_words(row.paragraph)
        df.loc[index, 'rare_words_score'] = get_rare_words_score(row.paragraph, row.question)

build features for val: 100%|██████████| 1000/1000 [00:06<00:00, 156.88it/s]


In [4]:
idfs = calculate_idfs(dftrain)

calc idf: 100%|██████████| 9062/9062 [00:12<00:00, 724.23it/s]


In [265]:
for name, df in [('train', dftrain), ('test', dftest), ('val', dfval)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):
        question = uniq_words(row.question)
        paragraph = uniq_words(row.paragraph)
        df.loc[index, 'len_paragraph'] = len(paragraph)
        df.loc[index, 'len_question'] = len(question)
        df.loc[index, 'len_intersection'] = len(paragraph & question)
        df.loc[index, 'idf_question'] = np.sum([idfs.get(word, 0.0) for word in question])
        df.loc[index, 'idf_paragraph'] = np.sum([idfs.get(word, 0.0) for word in paragraph])
        df.loc[index, 'idf_intersection'] = np.sum([idfs.get(word, 0.0) for word in paragraph & question])
        df.loc[index, 'convolve_score'] = get_convolve_score(row.paragraph, row.question)
        df.loc[index, 'rare_words_score'] = get_rare_words_score(row.paragraph, row.question)

build features for val: 100%|██████████| 1000/1000 [00:09<00:00, 101.71it/s]


In [172]:
dftrain.head(20)

,paragraph_id,question_id,paragraph,question,target,len_paragraph,len_question,len_intersection,idf_question,idf_paragraph,idf_intersection
0,1094,46273,"В отличие от рыб, земноводные (амфибии) и прес...",С какого года Русское Царство перешло на летои...,0.0,65.0,19.0,4.0,61.147393,276.872975,2.563939
1,7414,19164,В 1049 году Балдуину V удалось отнять у Герман...,Кто упомянул о его первых разногласиях со Штей...,0.0,78.0,31.0,4.0,130.467286,340.304312,3.064391
2,6744,39767,Стремление достичь предельных значений ёмкости...,Как называется имеющая мировое значение эпоха ...,0.0,58.0,20.0,5.0,63.454290,250.442841,5.788364
3,7300,36318,Первый практически пригодный двухтактный газов...,Что усугублялось из-за международного давления...,0.0,62.0,14.0,3.0,45.780231,271.774961,0.944176
4,7077,41534,Требуя от художника углубленного изучения изоб...,Какой характер носят пророчества Леонардо да В...,0.0,85.0,7.0,4.0,32.919097,371.141486,20.729693
5,3559,62585,Белки — высокомолекулярные органические вещест...,Какие действия предприняла подводная лодка Чер...,0.0,64.0,16.0,4.0,63.803913,226.961762,6.116762
6,4350,3730,Прайсинговые методы — в основе лежит принцип и...,"Как называют остановки, до которых и на которы...",0.0,78.0,12.0,5.0,33.959472,295.843089,3.073182
7,8012,86629,Применяли изначально для определения близкород...,Какой признак киевский монах Нестор-летописец ...,1.0,101.0,10.0,8.0,46.478067,406.091761,37.889658
8,3634,69421,Успешная конверсия по-разному определяется гру...,"Чтобы понять, по какому рекламному каналу приш...",1.0,80.0,18.0,17.0,59.652443,341.256510,58.572958
9,4350,27335,Прайсинговые методы — в основе лежит принцип и...,Урожайность сои в каких странах почти не отлич...,0.0,78.0,11.0,4.0,31.950789,295.843089,3.014817


In [385]:
columns = ['len_paragraph', 'len_question', 'len_intersection', 'idf_question', 'idf_paragraph', 'idf_intersection', 'convolve_score', 'rare_words_score']
model = RandomForestClassifier().fit(dftrain[columns], dftrain['target'])
dftest['prediction'] = model.predict(dftest[columns])

In [386]:
dftest[['paragraph_id', 'question_id', 'prediction']].to_csv("prediction.csv", index=False)

In [387]:
from sklearn.metrics import accuracy_score

predict_val = model.predict(dfval[columns])
accuracy_score(y_true=list(dfval['target']), y_pred=predict_val, normalize=False)

971

In [388]:
from sklearn.metrics import roc_auc_score
roc_auc_score(list(dfval['target']), predict_val)

0.96836514150558684